In [16]:
import folium
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from data_api import *
from utils.folium_utils import *

In [17]:
bus_ids = [51, 50, 31, 32, 4, 38, 30, 26, 13, 49, 40, 6, 29, 20, 10, 41, 3, 58, 7]

In [18]:
routes_gj = get_routes_geojson(bus_ids)
time_range_lis = [
    ('weekday', '08:00', '09:00'),
    ('weekday', '09:00', '10:00'),
    ('weekday', '10:00', '11:00')
]
# grid_gj = get_grid_geojson(bus_ids, time_range, stations_source='here')

In [8]:
m = folium.Map(location=[55.6795535, 12.542231], zoom_start=13, tiles='OpenStreetMap')

In [9]:
routes = folium.GeoJson(routes_gj, name='routes', style_function=lambda feature: {'color': 'black'})
grid = folium.GeoJson(grid_gj, name='grid', style_function=lambda feature: {
    'fillColor': feature['properties']['fillColor'],
    'color':"black",
    'weight': 2,
    'dashArray': '5, 5',
    'fillOpacity': 0.55,
})
grid.add_to(m)
routes.add_to(m)

NameError: name 'grid_gj' is not defined

In [23]:
folium.features.GeoJsonTooltip(fields=['buses_count_subset', 'buses_count_total'], 
                               aliases=['subset', 'total'], sticky=False).add_to(grid)
folium.features.GeoJsonTooltip(fields=['bus_id', 'name'], 
                               aliases=['id', 'bus']).add_to(routes)
folium.LayerControl().add_to(m)

In [19]:
def get_grid_geojson_redux(time_range):
    time_range = time_range_lis[time_range]
    grid = get_grid_geojson(bus_ids, time_range, stations_source='here')
    return grid

def get_map_from_grid(time_range):
    grid_gj = get_grid_geojson_redux(time_range)
    m = folium.Map(location=[55.6795535, 12.542231], zoom_start=13, tiles='OpenStreetMap')
    routes = folium.GeoJson(routes_gj, name='routes', style_function=lambda feature: {'color': 'black'})
    grid = folium.GeoJson(grid_gj, name='grid', style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color':"black",
        'weight': 2,
        'dashArray': '5, 5',
        'fillOpacity': 0.55,
    })
    grid.add_to(m)
    routes.add_to(m)
    folium.features.GeoJsonTooltip(fields=['buses_count_subset', 'buses_count_total'], 
                               aliases=['subset', 'total'], sticky=False).add_to(grid)
    folium.features.GeoJsonTooltip(fields=['bus_id', 'name'], 
                                aliases=['id', 'bus']).add_to(routes)
    folium.LayerControl().add_to(m)
    return m

grid = interact(get_map_from_grid, time_range=widgets.IntSlider(min=0, max=2, step=1, value=0))
# m = get_map_from_grid(grid)
# m

interactive(children=(IntSlider(value=0, description='time_range', max=2), Output()), _dom_classes=('widget-in…